In [ ]:
slurmHeader = lambda job_name,array_vals: f"""#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --mem=16Gb
#SBATCH --time=23:00:00
#SBATCH --cpus-per-task=16 # 16 threads per task
#SBATCH --partition=short
#SBATCH --output="output/{job_name}.%A.%a.out"
#SBATCH --error="output/{job_name}.%A.%a.err"
#SBATCH --array={array_vals}
eval "$(conda shell.bash hook)"
conda activate st2
cd /home/zeiberg.d/LEVERAGING_STRUCTURE_CODE_APPENDIX/
"""

In [ ]:
print(slurmHeader("test",",".join([str(s) for s in [1,3,5,7]])))

In [ ]:
acsRunCommand = lambda problem_name,problem_setting: f"srun python runExperiment.py --root_dir /scratch/zeiberg.d/folktables/ \
--experimentPath /scratch/zeiberg.d/leveragingStructureFinalExperiments/experiments/{problem_name}_setting_{problem_setting}_$SLURM_ARRAY_TASK_ID \
--datasetType acs --problem_setting {problem_setting} --problem_name {problem_name}"

In [ ]:
amazonRunCommand = lambda problem_setting: f"srun python runExperiment.py \
--experimentPath /scratch/zeiberg.d/leveragingStructureFinalExperiments/experiments/amazon_reviews_setting_{problem_setting}_$SLURM_ARRAY_TASK_ID \
--datasetType huggingface --problem_name amazon_reviews_multi --pca True"

In [ ]:
syntheticRunCommand = lambda problem_setting, syntheticDim, syntheticNClusters: f"""srun python runExperiment.py \
--experimentPath /scratch/zeiberg.d/leveragingStructureFinalExperiments/experiments/synthetic_dim_{syntheticDim}_nClusters_{syntheticNClusters}_setting_{problem_setting}_$SLURM_ARRAY_TASK_ID \
--datasetType synthetic --problem_name synthetic --problem_setting {problem_setting} --syntheticDim {syntheticDim} \
--syntheticNClusters {syntheticNClusters}"""

In [ ]:
syntheticExperimentParameterGrid = {
    "problem_setting" : [1,2],
    "syntheticDim" : [1,4,16,64],
    "syntheticNClusters" : [1,2,4,8],
}

In [ ]:
missingDict = {(1, 4, 1): '16,18,15,17,19,20',
 (4, 1, 1): '1,2',
 (16, 2, 1): '6,5',
 (16, 8, 1): '14,12,18,17,23,15,22,19,24,21,11,20,16,13',
 (64, 1, 1): '24',
 (64, 2, 1): '2',
 (1, 8, 2): '23',
 (16, 4, 2): '24',
 (16, 8, 2): '16,8,7,17,15,10,11,19,13,6,14,9,12,18',
 (64, 1, 2): '1,0',
 (64, 8, 2): '9'}

In [ ]:
from itertools import product

In [ ]:
mkdir -p scripts/slurmScriptsMissing/output

In [ ]:
for (problem_setting,syntheticDim,syntheticNClusters) in product(*list(syntheticExperimentParameterGrid.values())):
    if (syntheticDim,syntheticNClusters,problem_setting) in missingDict:
        with open(f"scripts/slurmScriptsMissing/run_synthetic_dim_{syntheticDim}_nClusters_{syntheticNClusters}_setting_{problem_setting}.sh","w") as f:
            f.write(slurmHeader(f"synthetic_dim_{syntheticDim}_nClusters_{syntheticNClusters}_setting_{problem_setting}",
                               missingDict[(syntheticDim,syntheticNClusters,problem_setting)]))
            f.write(syntheticRunCommand(problem_setting,syntheticDim, syntheticNClusters))

In [ ]:
acsParameterGrid = {
    "problem_setting" : [1,2],
    "problem_name" : ["income", "income_poverty_ratio","employment"],
}

In [ ]:
for (problem_setting,problem_name) in product(*list(acsParameterGrid.values())):
    if problem_name == "income_poverty_ratio":
        nums = "11,12,13,14"
    elif problem_name == "income":
        nums = "21"
    else:
        continue
    with open(f"scripts/slurmScriptsMissing/run_{problem_name}_setting_{problem_setting}.sh","w") as f:
        f.write(slurmHeader(f"{problem_name}_setting_{problem_setting}",nums))
        f.write(acsRunCommand(problem_name,problem_setting))

In [ ]:
for problem_setting in [1,2]:
    with open(f"scripts/slurmScripts/run_amazon_reviews_multi_setting_{problem_setting}.sh","w") as f:
        f.write(slurmHeader(f"amazon_reviews_multi_setting_{problem_setting}"))
        f.write(amazonRunCommand(problem_setting))